In [1]:
# Draws a bounding box around an object found in image
def drawRectangle(read_image, coordinates):
    # Represent all sides of a box
    image = Image.open(read_image)
    draw = ImageDraw.Draw(image)
    draw.rectangle(coordinates, outline='red')
    image.show()

In [5]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
# </snippet_imports>

'''
Authenticate
Authenticates your credentials and creates a client.
'''
# <snippet_vars>
subscription_key = "d5d0e372874a4c11ba463a30abe853f1"
endpoint = "https://answer-sheet.cognitiveservices.azure.com/"
# </snippet_vars>
# </snippet_imports_and_vars>

# <snippet_client>
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [6]:
from PIL import Image, ImageDraw
read_image = open("image0.png", "rb")
# image = Image.open(read_image)

# draw = ImageDraw.Draw(image)
# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(10)

image = Image.open(read_image)
draw = ImageDraw.Draw(image)



# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            
            print(line.text)
            #print(line.bounding_box)
            coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
            draw.rectangle(coordinates, outline='red')
image.show()
print()

ComputerVisionOcrErrorException: Operation returned an invalid status code 'PermissionDenied'